## Examples Setup
The following steps walk you through the steps needed to create the user for the demo and enable the schema for ords. **NOTE :** You should have installed ORDS against your target database, and it should running, before executing the following steps.

In [ ]:
import oracledb
from HelperFunctions import execute_plsql

%reload_ext sql

**UPDATE the following cell with the details of the username and password for a user with privileges to create users i.e. ```SYS```. and then the details of the user you want to create** 

In [ ]:
# Provide the details of the database user with DBA prvilege
sys_username = 'sys'
sys_password = 'welcome1'

# The connection string of your target database
db_connect_string = '//192.168.86.235/soe'

# The username of the user to run the demo from
user_name = 'ora23c'
user_password = 'ora23c'


The rest of the cells create the user and grants them the necessary privileges to run the examples.

In [ ]:
dba_connection = oracledb.connect(user=sys_username, password=sys_password, dsn=db_connect_string, mode=oracledb.AUTH_MODE_SYSDBA)
cursor = dba_connection.cursor() 
cursor.execute(f'DROP USER IF EXISTS {user_name} CASCADE ')
cursor.execute(f'CREATE USER {user_name} IDENTIFIED BY {user_password} ')
cursor.execute(f"ALTER USER {user_name} DEFAULT TABLESPACE USERS QUOTA UNLIMITED ON USERS")
cursor.execute(f"ALTER USER {user_name} TEMPORARY TABLESPACE TEMP")
cursor.execute(f"GRANT CONNECT, RESOURCE, DB_DEVELOPER_ROLE, CREATE MLE to {user_name}")
cursor.execute(f"GRANT EXECUTE ON JAVASCRIPT to {user_name}")

Once the user has been created it then enables the user for REST access by Oracle ORDS

In [ ]:
user_connection = oracledb.connect(user=user_name, password=user_password, dsn=db_connect_string)

In [ ]:
cursor = user_connection.cursor()
cursor.execute("select * from all_objects where object_name = 'ORDS'")
result_set = cursor.fetchmany()
if len(result_set) > 0:
    cursor.execute(f'''
        begin
            ORDS.enable_schema(
                p_enabled             => TRUE,
                p_schema              => '{user_name}',
                p_url_mapping_type    => 'BASE_PATH',
                p_url_mapping_pattern => '{user_name}',
                p_auto_rest_auth      => TRUE
              );
        end;''')
    cursor.execute(f'''
        BEGIN
            ORDS.ENABLE_OBJECT(
                    p_object => 'CUSTOMERS_DV',
                    p_object_type => 'VIEW'
                );
        END;''')
else:
    print("ORDS hasn't been installed in the database")

### Drop user
The following code will drop the user created above

In [ ]:
dba_connection = oracledb.connect(user=sys_username, password=sys_password, dsn=db_connect_string, mode=oracledb.AUTH_MODE_SYSDBA)

In [ ]:
cursor.execute(f'DROP USER IF EXISTS {user_name} CASCADE ')